In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import csv
import seaborn as sns

In [2]:
tweets=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1", names=["target", "ids", "date", "flag", "user", "text"])
tweets.drop(['flag','date','user'],axis=1,inplace=True)
tweets['target']=tweets['target'].apply(lambda x: x/4)

In [3]:
tweets['target'].value_counts()

1.0    800000
0.0    800000
Name: target, dtype: int64

In [4]:
#tweets['target'] = tweets['target'].replace(1, 'positive')
#tweets['target'] = tweets['target'].replace(0, 'negative')

In [5]:
#tweets = np.random.shuffle(tweets.values)

In [6]:
#tweets= tweets.iloc[0:100,:]

In [7]:
#PREPROCESSING USING REGULAR EXPRESSION
def clean_tweets(tweet):    
    # remove URL
    tweet = re.sub(r"http\S+", "", tweet)    
    # Remove usernames
    tweet = re.sub(r"@[^\s]+[\s]?",'',tweet)    
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)    
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)    
    tweet = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
    tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
    tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", tweet)
    tweet = tweet.lower()
    tweet = re.sub(r"that's","that is",tweet)
    tweet = re.sub(r"there's","there is",tweet)
    tweet = re.sub(r"what's","what is",tweet)
    tweet = re.sub(r"where's","where is",tweet)
    tweet = re.sub(r"it's","it is",tweet)
    tweet = re.sub(r"who's","who is",tweet)
    tweet = re.sub(r"i'm","i am",tweet)
    tweet = re.sub(r"she's","she is",tweet)
    tweet = re.sub(r"he's","he is",tweet)
    tweet = re.sub(r"they're","they are",tweet)
    tweet = re.sub(r"who're","who are",tweet)
    tweet = re.sub(r"ain't","am not",tweet)
    tweet = re.sub(r"wouldn't","would not",tweet)
    tweet = re.sub(r"shouldn't","should not",tweet)
    tweet = re.sub(r"can't","can not",tweet)
    tweet = re.sub(r"couldn't","could not",tweet)
    tweet = re.sub(r"won't","will not",tweet)
    tweet = re.sub(r"\W"," ",tweet)
    tweet = re.sub(r"\d"," ",tweet)
    tweet = re.sub(r"\s+[a-z]\s+"," ",tweet)
    tweet = re.sub(r"\s+[a-z]$"," ",tweet)
    tweet = re.sub(r"^[a-z]\s+"," ",tweet)
    tweet = re.sub(r"\s+"," ",tweet)    
    return tweet

In [8]:
# Apply function to Tweet column

tweets['text'] = tweets['text'].apply(clean_tweets)

In [1]:
nltk.download('stopwords')

NameError: name 'nltk' is not defined

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [11]:
#nltk.download()

In [12]:
#STOP WORDS REMOVAL
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

tweets['STOPWORD']=tweets['text'].apply(lambda x:' '.join([word for word in x.split()if word not in (stops)]))


In [13]:
##SLANG REMOVAL
def translator(user_string):
    user_string = user_string.split(" ")
    
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "C:/Users/Lenovo/Desktop/sentiment analysis_update/slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its Abbreviation in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return(' '.join(user_string))
   

In [ ]:
 tweets['Cleaned']=tweets['STOPWORD'].apply(translator)

In [ ]:
col_list=['target','ids','text','STOPWORD','Cleaned']

In [ ]:
tweets=tweets[col_list]

In [ ]:
#!pip install vadersentiment

In [ ]:
from nltk.corpus import sentiwordnet as swn
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def analyze_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    else:
        return 'negative'


In [ ]:
tweets['sentiment']=tweets['Cleaned'].apply(analyze_sentiment)

In [ ]:
sid=SentimentIntensityAnalyzer()

In [ ]:
tweets['VaderValues']=tweets['text'].apply(lambda x: sid.polarity_scores(x))


In [ ]:
tweets['VaderScore']=tweets['VaderValues'].apply(lambda x: x['compound'])

In [ ]:
tweets['VaderScore']=tweets['VaderScore'].apply(lambda score: 'positive' if score >= 0.05 else('negative')) 

In [ ]:
tweets=tweets.drop(columns='VaderValues', axis=0)

In [ ]:
sns.countplot(x='VaderScore',data=tweets) # vander sentiments

In [ ]:
sns.countplot(x='sentiment',data=tweets) # text blob

In [ ]:
##labels for vader sentiments
features = tweets.iloc[:, 3].values
labels = tweets.iloc[:,6].values

In [ ]:
tweets.columns

In [ ]:
tweets.groupby(['target','VaderScore']).count()

In [ ]:
tweets.groupby(['target','sentiment']).count()

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2865,min_df=3, max_df=0.6, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(features).toarray()


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.3, random_state=100)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=100)
text_classifier.fit(X_train, y_train)

In [ ]:
predictions = text_classifier.predict(X_test)

In [ ]:
predictions

In [ ]:
predictions

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

In [ ]:
accuracy_rf = accuracy_score(y_test,predictions)
print("Accuracy: {}".format(accuracy_rf))

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
##labels for TextBlob Sentiments
features2 = tweets.iloc[:, 3].values
labels2 = tweets.iloc[:,5].values

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer1 = TfidfVectorizer (max_features=2865,min_df=3, max_df=0.6, stop_words=stopwords.words('english'))
processed_features1 = vectorizer.fit_transform(features2).toarray()

In [ ]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(processed_features1, labels2, test_size=0.3, random_state=25)

# Naive Bayes

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB


In [ ]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train1, y_train1)

#Predict the response for test dataset
y_pred = gnb.predict(X_test1)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test1, y_pred))

In [ ]:
print(classification_report(y_test1,y_pred))

# SVM

In [ ]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train1, y_train1) 
lin_clf_pred = lin_clf.predict(X_test1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test1,lin_clf_pred))
print(classification_report(y_test1,lin_clf_pred))
print(accuracy_score(y_test1, lin_clf_pred))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model = log_model.fit(X_train1, y_train1)

In [ ]:
y_pred1 = log_model.predict(X_test1)

In [ ]:
accuracy_lr = accuracy_score(y_test1,y_pred1)
print("Accuracy: {}".format(accuracy_lr))

In [ ]:


print(confusion_matrix(y_test1,y_pred1))
print(classification_report(y_test1,y_pred1))
print(accuracy_score(y_test1, y_pred1))

# Multi Nomial Logistic Regression

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train1, y_train1)
clf_pred1 = clf.predict(X_test1)

In [ ]:
print(confusion_matrix(y_test1,clf_pred1))
print(classification_report(y_test1,clf_pred1))
print(accuracy_score(y_test1, clf_pred1))

# Ensembling -- XGBOOST

In [ ]:
from xgboost import XGBClassifier
# fit model no training data
model_XGB = XGBClassifier(n_estimators=200, max_depth=100, learning_rate=0.01)
model_XGB.fit(X_train1, y_train1)

In [ ]:
model_XGB_pred1 = model_XGB.predict(X_test1)

In [ ]:
print(confusion_matrix(y_test1,model_XGB_pred1))
print(classification_report(y_test1,model_XGB_pred1))
print(accuracy_score(y_test1, model_XGB_pred1))